In [10]:
from Bio import SeqIO

rna_dict = {}
protein_dict = {}

# RefSeq
ref_seq_acc = {}
with open('hg38_refseq_metadata.txt', "r") as handle:
    for line in handle.readlines()[1:]:
        tokens = line.split('\t')
        ref_seq_acc[tokens[5]] = tokens[4]
        
with open('GRCh38_latest_cds.fna', "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        header = record.description
        start_pos = header.find("protein_id=")
        end_pos = header[start_pos:].find("]")
        protein_id = header[start_pos:][:end_pos].split('=')[-1].strip()
        try:
            transcript_id = ref_seq_acc[protein_id]
            rna_dict[transcript_id] = str(record.seq).strip()
        except KeyError:
            pass
    
with open('GRCh38_latest_protein.faa', "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        protein_id = record.description.split()[0].strip()
        try:
            transcript_id = ref_seq_acc[protein_id]
            protein_dict[transcript_id] = str(record.seq).strip()
        except KeyError:
            pass
        
# Ensembl
with open('Homo_sapiens.GRCh38.cds.all.fa', "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        transcript_id = record.description.split()[0].strip()
        rna_dict[transcript_id] = str(record.seq).strip()
    
with open('Homo_sapiens.GRCh38.pep.all.fa', "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        transcript_id = record.description.split()[4].split(':')[1]
        protein_dict[transcript_id] = str(record.seq).strip()
        
# UCSC
with open('knownGene_cds.fna', "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        transcript_id = record.description.split()[0].split("_")[-1].strip()
        rna_dict[transcript_id] = str(record.seq).strip()
    
with open('knownGene_protein.faa', "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        transcript_id = record.description.split()[0].strip()
        protein_dict[transcript_id] = str(record.seq).strip()
        
# Write dictionaries
with open('hg38_cDNA_DICT.dict', 'w') as handle:
    for key,value in rna_dict.items():
        handle.write("{}:{}\n".format(key,value))
    
with open('hg38_PROTEIN_DICT.dict', 'w') as handle:
    for key,value in protein_dict.items():
        handle.write("{}:{}\n".format(key,value))

In [17]:
anno_var = set()
with open("hg38_ensGene.txt", "r") as handle:
    for line in handle.readlines():
        anno_var.add(line.split("\t")[1])
        
# Ensembl
gencode = set()
with open('gencode.v35.pc_transcripts.fa', "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        gencode.add(record.description.split("|")[0])

In [18]:
print(len(anno_var))
print(len(gencode))

89663
101486


In [19]:
test1 = {}
with open('hg38_PROTEIN_DICT.dict', 'r') as handle:
    for line in handle.readlines():
        tokens = line.split(":")
        test1[tokens[0]] = tokens[1]

len(list(set(test1.keys()).intersection(anno_var)))

46447

In [20]:
len(gencode.intersection(anno_var))

38388